In [1]:
import wandb
import os

## Login W&B

In [2]:
wandb.login()

wandb: Currently logged in as: deepchecks (use `wandb login --relogin` to force relogin)


True

In [3]:
PROJECT_NAME = "dc-wandb-webinar"
ENTITY_NAME = "deepchecks"

## Load Avocado Data

### Get data and save metadata

In [4]:
from deepchecks.tabular import datasets
import avocado_dataset_utils

data = datasets.regression.avocado.load_data(data_format='DataFrame', as_train_test=False)

# a bit preprocessing
train_df, test_df = avocado_dataset_utils.get_train_test_df_from_raw(data)
label_col_name = 'IsExpensive'
categorical_features = ['type']

### Create Dataset

So checks have metadata context (label, categorical feature, date column if exists, etc.)

In [5]:
from deepchecks.tabular import Dataset

train_ds = Dataset(train_df, label=label_col_name, cat_features=categorical_features)
test_ds = Dataset(test_df, label=label_col_name, cat_features=categorical_features)

## Check Data Integrity

In [6]:
from deepchecks.tabular.suites import single_dataset_integrity

integ_suite_results = single_dataset_integrity().run(train_ds)
integ_suite_results

C:\Users\Shir\Downloads\deepchecks-wandb-webinar\venv\lib\site-packages\deepchecks\suites.py:21: DeprecationWarning:

Ability to import tabular suites from the `deepchecks.suites` is deprecated, please import from `deepchecks.tabular.suites` instead



Single Dataset Integrity Suite:   0%|          | 0/8 [00:00<?, ? Check/s]

In [7]:
integ_suite_results.to_wandb(project=PROJECT_NAME, entity=ENTITY_NAME, name="initial-data-integrity")

Single Dataset Integrity Suite:   0%|          | 0/8 [00:00<?, ? Result/s]

### add noise to data

In [8]:
# add data duplicates, ambiguous labels, string mismatch, ...
dirty_train_df = avocado_dataset_utils.add_dirty_data_to_single_df(train_df)
dirty_train_ds = Dataset(dirty_train_df, cat_features = categorical_features, label = label_col_name)

### Rerun integrity suite

In [9]:
dirty_integ_suite_results = single_dataset_integrity().run(dirty_train_ds)
dirty_integ_suite_results

Single Dataset Integrity Suite:   0%|          | 0/8 [00:00<?, ? Check/s]

#### Log suite results to w&b

In [10]:
dirty_integ_suite_results.to_wandb(project=PROJECT_NAME, entity=ENTITY_NAME, name="dirty-data-integrity")

Single Dataset Integrity Suite:   0%|          | 0/8 [00:00<?, ? Result/s]

#### Now lets log (in a new run) specifically checks that didn't pass

In [12]:
from deepchecks.core import CheckResult

# we want to have them together in one run, so lets init the run:
wandb.init(project=PROJECT_NAME, entity=ENTITY_NAME, name="integrity-not-passing-checks")

for check_result in dirty_integ_suite_results.results:
    if type(check_result) is CheckResult:
        if not check_result.passed_conditions():
            print("Check **{}** didn't Pass. Saving result to wandb".format(check_result.get_header()))
            # Save here to the same run:
            check_result.to_wandb(dedicated_run = False)
            
wandb.finish()

Check Single Value in Column Didn't Pass, logging to wandb
Check String Mismatch Didn't Pass, logging to wandb
Check Data Duplicates Didn't Pass, logging to wandb
Check Label Ambiguity Didn't Pass, logging to wandb



## Model

### Prepare Data

In [ ]:
train_df, train_ohe_columns = avocado_dataset_utils.ohe_for_type_column(train_df)
test_df, _ = avocado_dataset_utils.ohe_for_type_column(test_df, train_ohe_columns)

train_ds = Dataset(train_df, label=label_col_name, cat_features=[])
test_ds = Dataset(test_df, label=label_col_name, cat_features=[])

### Fit & Predict Model

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# fit
rf_clf = RandomForestClassifier()
rf_clf.fit(train_ds.features_columns, train_ds.label_col)

In [ ]:
# predict
y_pred_train = rf_clf.predict(train_ds.features_columns)
y_pred_test = rf_clf.predict(test_ds.features_columns)

In [ ]:
from sklearn.metrics import accuracy_score

# minimal metrics check - model accuracty
train_acc = accuracy_score(train_ds.label_col, y_pred_train)
test_acc = accuracy_score(test_ds.label_col, y_pred_test)
print("Train accuracy: {}, Test accuracy: {}".format(train_acc, test_acc))

### Analyze Splits and Evaluate

#### Model Evaluation Suite

In [ ]:
from deepchecks.tabular.suites import model_evaluation

me_suite_result = model_evaluation().run(train_ds, test_ds, rf_clf)
me_suite_result

#### Run performance report and log

In [ ]:
from deepchecks.tabular.checks import PerformanceReport

perf_result = PerformanceReport().run(train_ds, test_ds, rf_clf)

In [ ]:
# show output value
perf_result.value

In [ ]:
# show 
perf_result.show()

In [ ]:
perf_result.to_wandb(project=PROJECT_NAME, entity=ENTITY_NAME, name="performance-report")

#### Inspect Split

In [ ]:
from deepchecks.tabular.suites import train_test_validation

train_test_validation().run(train_ds, test_ds, rf_clf)

### Run specific checks - separately or in a suite

In [ ]:
from deepchecks.tabular.checks import ConfusionMatrixReport, CalibrationScore, DataDuplicates
from deepchecks.tabular import Suite

In [ ]:
custom_suite = Suite('Custom Evaluation', CalibrationScore(), ConfusionMatrixReport())
suite_res = custom_suite.run(train_ds, test_ds, rf_clf)
suite_res

In [ ]:
suite_res.to_wandb(project=PROJECT_NAME, entity=ENTITY_NAME, name="custom-model-eval")

## Happy, Evaluated and Valid Models and Data...

The End

In [ ]:
# original utils (moved to file)

In [ ]:
# # minimal preprocessing and data splitting - move to utils file....
# import pandas as pd

# def get_classification_label_from_avg_price(df):
#     EXPENSIVE_AVOCADO_AVG_PRICE = 1.6
#     df['IsExpensive'] = df['AveragePrice'] > EXPENSIVE_AVOCADO_AVG_PRICE
#     df = df.drop(columns='AveragePrice')
#     return df

# def drop_unused_columns(df):
#     df = df.drop(columns=['Unnamed: 0'])
#     df = df.drop(columns=['region', 'Date'])
#     return df

# def organize_df(df):
#     df = df.sort_values(by='Date').reset_index(drop=True)
#     df = get_classification_label_from_avg_price(df)
#     df = drop_unused_columns(df)
#     return df

# def ohe_for_type_column(df, org_dummies_columns=None):
#     # there are only two values for type, getting dummies for all dataframe together
#     generated_columns = None
#     if 'type' in df.columns:
#         dummies = pd.get_dummies(df['type'])
#         generated_dummy_columns = dummies.columns
#         if org_dummies_columns is not None:
#             columns_only_in_org = set(org_dummies_columns) - set(generated_dummy_columns)
#             if columns_only_in_org:
#                 # append also columns existant in previous
#                 dummies[list(columns_only_in_org)] = 0
#             columns_only_in_result = set(generated_dummy_columns) - set(org_dummies_columns)
#             if columns_only_in_result:
#                 dummies = dummies.drop(columns=list(columns_only_in_result))
#             # set order to be similar
#             dummies = dummies[org_dummies_columns]
#         df = pd.concat([df, dummies], axis=1)
#         df = df.drop(columns='type')
#     return df, generated_dummy_columns

# def get_train_test_df_from_raw(df, train_frac=0.7):
#     # nothing here special to train/test so doing it together
#     df = organize_df(df)
    
#     # data is chronological, so split should be accordingly
#     train_samples = int(len(df)*train_frac)
#     train_df = df[:train_samples]
#     test_df = df[train_samples:]
#     return train_df, test_df

In [ ]:
# # make data dirty - move to utils file...

# import pandas as pd
# import numpy as np


# # def add_something_bad_to_model():
# #     # how?? dunno... but can show comparison...
# #     pass

# def add_organic_string_mismatch(df):
#     # copying since changes are inplace. returning df for consistency
#     df_dirty = df.copy(deep=True)
#     df_dirty.loc[df[df['type'] == 'organic'].sample(frac=0.18).index,'type'] = 'Organic'
#     df_dirty.loc[df[df['type'] == 'organic'].sample(frac=0.01).index,'type'] = 'ORGANIC'
#     return df_dirty


# def add_duplicates_with_shuffled_labels(df, label_col_name='IsExpensive'):
#     frac_to_duplicate = 0.36
#     labels = df[label_col_name]
#     duplicated_samples = df.sample(frac=frac_to_duplicate)
#     # shuffle labels, will likely add ambiguity
#     duplicated_samples[label_col_name] = duplicated_samples[label_col_name].sample(frac=1, ignore_index=True)
#     df_dup_labels = pd.concat([df, duplicated_samples], ignore_index=True)
#     return df_dup_labels
    

# def add_data_duplicates(df, frac_to_duplicate=0.156):
#     duplicated_samples = df.sample(frac=frac_to_duplicate)
#     df_dup = pd.concat([df, duplicated_samples], axis=0, ignore_index=True)
#     return df_dup

    
# def add_single_value(df):
#     new_column_df = df.copy(deep=True)
#     new_column_df['Is Ripe'] = True
#     return new_column_df


# def add_test_sample_leakage_to_train(train_df, test_df):
#     train_df = pd.concat([test_df.sample(frac=0.03), train_df], ignore_index=True)
#     return train_df


# def add_dirty_data_to_single_df(df):
#     df = add_organic_string_mismatch(df)
#     df = add_duplicates_with_shuffled_labels(df)
#     df = add_data_duplicates(df)
#     df = add_single_value(df)
#     return df


# def add_regression_drift_to_test_labels(test_df, label_col_name = 'AveragePrice'):
#     test_df[label_col_name] = test_df[label_col_name] + np.random.rand(test_df.shape[0])*test_df[label_col_name].std()*4
#     return test_df